In [ ]:
import json
import numpy as np
import xobjects as xo
import xtrack as xt
import xpart as xp
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import constants 
####################
# Choose a context #
####################
context = xo.ContextCpu()
# context = xo.ContextCpu(omp_num_threads='auto')
buf = context.new_buffer()

#references: https://www.sciencedirect.com/science/article/pii/S0168900222011445?ref=pdf_download&fr=RR-2&rr=80ca25af8a5ace93

# Ion properties:
m_u = 931.49410242e6 # eV/c^2 -- atomic mass unit
A = 16 # Weight of O
#Z = 6  # Number of protons in the ion (O)
#m_e = 0.511e6 # eV/c^2 -- electron mass
m_p = 938.272088e6 # eV/c^2 -- proton mass
clight = 299792458.0 # m/s

q0=5

mass0 = A*m_u #+ Ne*m_e # eV/c^2

beta_rel = 0.64
gamma_rel = 1.30

p0c = mass0*beta_rel*gamma_rel #eV/c

# equiv_proton_momentum = 236e9 # eV/c = gamma_p*m_p*v
# gamma_p = np.sqrt( 1 + (equiv_proton_momentum/m_p)**2 ) # equvalent gamma for protons in the ring

# p0c = equiv_proton_momentum*(Z-Ne) # eV/c
gamma2 = np.sqrt( 1 + (p0c/mass0)**2 ) # ion relativistic factor
beta2 = np.sqrt(1-1/(gamma2*gamma2)) # ion beta

print(gamma2)
print(beta2)


In [ ]:
mass0

In [ ]:
circumference =  128.80 #m
T = circumference/(clight*beta_rel)
s_per_turn = T
slip_factor=0.45

beta_x = 6
beta_y = 2

disp_x = 0
Q_x = 2.2
Q_y = 2.4
dQx = 0
dQy = 0

arc = xt.LineSegmentMap(
        qx=Q_x, qy=Q_y,
        dqx=dQx, dqy=dQy,
        length=circumference,
        betx=beta_x,
        bety=beta_y
        )

In [ ]:
s_per_turn

In [ ]:
emittance_x=10*1e-6 #inital emittance
emittance_y=15*1e-6 #inital emittance
num_particles = int(1e4)

sigma_x = np.sqrt(beta_x*emittance_x)
sigma_px = np.sqrt(emittance_x*1/beta_x)
sigma_y = np.sqrt(beta_y*emittance_y)
sigma_py = np.sqrt(emittance_y*1/beta_y)
sigma_p = 2e-5 # relative ion momentum spread
sigma_p = 2e-4 # relative ion momentum spread

delta = np.random.uniform(low=0.45e-4, high=0.65e-4, size=num_particles)

delta = np.random.uniform(low=0e-4, high=1e-4, size=num_particles)

# delta = np.random.normal(loc=0, scale=sigma_p, size=num_particles)
x = np.random.normal(loc=0.0, scale=sigma_x, size=num_particles) + disp_x * delta
px = np.random.normal(loc=0.0, scale=sigma_px, size=num_particles)
y = np.random.normal(loc=0.0, scale=sigma_y, size=num_particles)
py = np.random.normal(loc=0.0, scale=sigma_py, size=num_particles)

particles = xp.Particles(
    mass0=mass0,
    p0c=p0c,
    q0=q0,
    x=x,
    px=px,
    y=y,
    py=py,
    delta=delta,
    zeta=0
)

print('sigma_px',sigma_px*1e3)

In [ ]:
##################
# Laser Cooler #
##################

#laser-ion beam collision angle
theta_l = 2.6*np.pi/180 # rad
theta_l = 0
nx = 0; ny = -np.sin(theta_l); nz = -np.cos(theta_l)

# Ion excitation energy:
# hw0 = 230.823 # eV
hc=constants.hbar*clight/constants.e # eV*m (ħc)
lambda_0 = 103.76*1e-9 # m -- ion excitation wavelength
hw0 = 2*np.pi*hc/lambda_0 # m -- ion excitation wavelength
ion_excited_lifetime=2.44e-9


lambda_l = lambda_0*gamma_rel*(1 + beta_rel*np.cos(theta_l)) # m -- laser wavelength
# Shift laser wavelength for fast longitudinal cooling:
#lambda_l = lambda_l*(1+3*sigma_p)
#lambda_l = lambda_l*(1+2.5*sigma_p)
lambda_l = 2.2131631631631633e-07
lambda_l = 2.213116311631163e-07
#lambda_l = lambda_l*(1+1*sigma_p) # m

laser_frequency = clight/lambda_l # Hz


print('Laser wavelength = %.2f nm' % (lambda_l/1e-9))
print('Laser wavelength = %.20f nm' % (lambda_l/1e-9))

# laser_waist_radius = 5*1e-3
# laser_waist_radius = 1000*1e-3

laser_power=40*1e-3 #W
laser_waist_radius = 20*1e-3
laser_area=np.pi*(laser_waist_radius*laser_waist_radius)

# laser_waist_radius = 1000*1e-3
# laser_intensity=200000
# laser_intensity=12732.395447351628
laser_intensity=laser_power/laser_area

cooling_section_length=25
GF_IP = xt.CWLaser(_buffer=buf,
                      laser_x=0,
                      laser_y=0,
                      laser_z=0,
                      
                      laser_direction_nx = 0,
                      laser_direction_ny = 0,
                      laser_direction_nz = -1,
                      laser_wavelength = lambda_l, # m
                      laser_waist_radius = laser_waist_radius, # m
                      laser_intensity=laser_intensity,
                      ion_excitation_energy = hw0, # eV
                      ion_excited_lifetime  = ion_excited_lifetime, # sec
                      cooling_section_length=cooling_section_length
                          
   )


In [ ]:
##################
# Tracking #
##################
x, px, zeta,accumulated_length,excited_list,delta_list,state_list= [], [], [], [], [],[],[]
num_turns = int(103.6*1e6) #Lanzhou turns = 103.6*1e6
save_interval=num_turns/1000
# loop over turns
for i in tqdm(range(num_turns)):
    if i % save_interval ==0:
    
        x.append(particles.x.copy())
        px.append(particles.px.copy())
        # zeta.append(particles.zeta)
        # accumulated_length.append(particles.s)

        state=particles.state.copy()
        state_list.append(state)

        excited=particles.state==2
        fraction_excitation = sum(excited)/len(excited)

        excited_list.append(excited)

        delta_list.append(particles.delta.copy())
    
    # track particle
    arc.track(particles)
    GF_IP.track(particles)

# #tracker=xt.Tracker(_context=context, _buffer=buf, line=line,reset_s_at_end_turn=False)

# # simulation parameters: simulate 10 s of cooling, and take data once every 100 ms
# max_time_s = 0.02
# int_time_s = 0.001
# T_per_turn = circumference/(clight*beta_rel)
# num_turns = int(max_time_s/T_per_turn)
# save_interval = int(int_time_s/T_per_turn)

# num_turns = int(1e6)
# save_interval=num_turns/100

# # create a monitor object, to reduce holded data
# monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=1,
#                               n_repetitions=int(num_turns/save_interval),
#                               repetition_period=save_interval,
#                               num_particles=num_particles)

# line = xt.Line(
#         elements=[monitor,GF_IP, arc])

# line.particle_ref = xp.Particles(mass0=mass0, q0=q0, p0c=p0c)
# #line.build_tracker(_context=context,reset_s_at_end_turn=False)


# tracker=xt.Tracker(_context=context, _buffer=buf, line=line)

# line.track(particles, num_turns=num_turns,
#               turn_by_turn_monitor=False)

# # extract relevant values
# x = monitor.x[:,:,0]
# px = monitor.px[:,:,0]
# y = monitor.y[:,:,0]
# py = monitor.py[:,:,0]
# delta = monitor.delta[:,:,0]
# zeta = monitor.zeta[:,:,0]
# accumulated_length=monitor.s[:,:,0]
# state = monitor.state[:,:,0]
# time = monitor.at_turn[:, 0, 0] * T_per_turn

# excited=particles.state==2
# excited = excited.astype(int)
# fraction_excitation = sum(excited)/len(excited)
time = np.arange(0, num_turns, save_interval) * s_per_turn

In [ ]:
# np.savez('results/langzhou_data.npz', x=x, px=px, y=y, py=py,accumulated_length=accumulated_length,zeta=zeta, delta=delta_list,
#  state=state_list, time=time,s_per_turn=s_per_turn,slip_factor=slip_factor)
np.savez('results/langzhou_data.npz', x=x, px=px, y=y, py=py,zeta=zeta, delta=delta_list, excited=excited_list,time=time)
np.savez(f'results/langzhou_data({laser_waist_radius*1e3}mm).npz', x=x, px=px, y=y, py=py,zeta=zeta, delta=delta_list, excited=excited_list,time=time)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.rcParams.update({'font.size': 25})

# Assuming 'fraction_excitation' is a variable that stores the fraction excited
plt.title(f'Fraction excited: {fraction_excitation*100:.5f}%')

# Other plotting commands
# plt.axvline(laser_x,color='red')
# plt.axvline(laser_x+laser_waist_radius,color='red')
# plt.axvline(laser_x-laser_waist_radius,color='red')

plt.scatter(particles.x*1e3, particles.px)
plt.scatter(particles.x[excited]*1e3, particles.px[excited])
plt.xlabel('x [mm]')
plt.ylabel('px')
plt.show()

In [ ]:
plt.figure()
plt.figure(figsize=(12, 8))
plt.rcParams.update({'font.size': 25})
plt.title(f'Fraction excited: {round(fraction_excitation*100, 5)}%')

#plt.axvline(laser_x,color='red')
# plt.axvline(laser_x+laser_waist_radius,color='red')
# plt.axvline(laser_x-laser_waist_radius,color='red')

plt.scatter(particles.x*1e3,particles.delta)
plt.scatter(particles.x[excited]*1e3,particles.delta[excited])
plt.xlabel('x [mm]')
plt.ylabel('dp/p ')
plt.tight_layout()

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
# plt.savefig('First_turn.eps', format='eps', dpi=300)
plt.show()